# Saving SCF results on disk and SCF checkpoints

For longer DFT calculations it is pretty standard to run them on a cluster
in advance and to perform postprocessing (band structure calculation,
plotting of density, etc.) at a later point and potentially on a different
machine.

To support such workflows DFTK offers the two functions `save_scfres`
and `load_scfres`, which allow to save the data structure returned
by `self_consistent_field` on disk or retrieve it back into memory,
respectively. For this purpose DFTK uses the
[JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file format and Julia package.

> **Availability of `load_scfres`, `save_scfres` and checkpointing**
>
> As JLD2 is an optional dependency of DFTK these three functions are only
> available once one has *both* imported DFTK and JLD2 (`using DFTK`
> and `using JLD2`).

To illustrate the use of the functions in practice we will compute
the total energy of the O₂ molecule at PBE level. To get the triplet
ground state we use a collinear spin polarisation
(see Collinear spin and magnetic systems for details)
and a bit of temperature to ease convergence:

In [1]:
using DFTK
using LinearAlgebra
using JLD2
using PseudoPotentialData

d = 2.079  # oxygen-oxygen bondlength
a = 9.0    # size of the simulation box
lattice = a * I(3)
pseudopotentials = PseudoFamily("cp2k.nc.sr.pbe.v0_1.semicore.gth")
O = ElementPsp(:O, pseudopotentials)
atoms     = [O, O]
positions = d / 2a * [[0, 0, 1], [0, 0, -1]]
magnetic_moments = [1., 1.]

Ecut  = 10  # Far too small to be converged
model = model_DFT(lattice, atoms, positions;
                  functionals=PBE(),
                  temperature=0.02, smearing=Smearing.Gaussian(),
                  magnetic_moments)
basis = PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])

scfres = self_consistent_field(basis, tol=1e-2, ρ=guess_density(basis, magnetic_moments))
save_scfres("scfres.jld2", scfres);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   |Magn|   Diag   Δtime
---   ---------------   ---------   ---------   ------   ------   ----   ------
  1   -27.64921878424                   -0.13    0.001    0.185    6.5    3.63s
  2   -28.92461936549        0.11       -0.83    0.694    0.698    2.0    2.57s
  3   -28.93121160527       -2.18       -1.13    1.200    1.210    2.5    108ms
  4   -28.93781335436       -2.18       -1.19    1.782    1.802    1.5   96.3ms
  5   -28.93919422759       -2.86       -1.03    2.000    2.060    2.0   98.4ms
  6   -28.93958683053       -3.41       -1.90    1.976    2.012    1.5   91.1ms
  7   -28.93961183516       -4.60       -3.10    1.985    2.023    1.0   83.8ms


In [2]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             16.7725575
    AtomicLocal         -58.4946185
    AtomicNonlocal      4.7085752 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3611122
    Xc                  -6.3913081
    Entropy             -0.0008790

    total               -28.939611835160

The `scfres.jld2` file could now be transferred to a different computer,
Where one could fire up a REPL to inspect the results of the above
calculation:

In [3]:
using DFTK
using JLD2
loaded = load_scfres("scfres.jld2")
propertynames(loaded)

(:α, :history_Δρ, :converged, :occupation, :occupation_threshold, :algorithm, :basis, :runtime_ns, :n_iter, :n_matvec, :history_Etot, :εF, :energies, :ρ, :timedout, :n_bands_converge, :τ, :eigenvalues, :ψ, :ham)

In [4]:
loaded.energies

Energy breakdown (in Ha):
    Kinetic             16.7725575
    AtomicLocal         -58.4946185
    AtomicNonlocal      4.7085752 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3611122
    Xc                  -6.3913081
    Entropy             -0.0008790

    total               -28.939611835160

Since the loaded data contains exactly the same data as the `scfres` returned by the
SCF calculation one could use it to plot a band structure, e.g.
`plot_bandstructure(load_scfres("scfres.jld2"))` directly from the stored data.

Notice that both `load_scfres` and `save_scfres` work by transferring all data
to/from the master process, which performs the IO operations without parallelisation.
Since this can become slow, both functions support optional arguments to speed up
the processing. An overview:
- `save_scfres("scfres.jld2", scfres; save_ψ=false)` avoids saving
  the Bloch wave, which is usually faster and saves storage space.
- `load_scfres("scfres.jld2", basis)` avoids reconstructing the basis from the file,
  but uses the passed basis instead. This save the time of constructing the basis
  twice and allows to specify parallelisation options (via the passed basis). Usually
  this is useful for continuing a calculation on a supercomputer or cluster.

See also the discussion on Input and output formats on JLD2 files.

## Checkpointing of SCF calculations
A related feature, which is very useful especially for longer calculations with DFTK
is automatic checkpointing, where the state of the SCF is periodically written to disk.
The advantage is that in case the calculation errors or gets aborted due
to overrunning the walltime limit one does not need to start from scratch,
but can continue the calculation from the last checkpoint.

The easiest way to enable checkpointing is to use the `kwargs_scf_checkpoints`
function, which does two things. (1) It sets up checkpointing using the
`ScfSaveCheckpoints` callback and (2) if a checkpoint file is detected,
the stored density is used to continue the calculation instead of the usual
atomic-orbital based guess. In practice this is done by modifying the keyword arguments
passed to # `self_consistent_field` appropriately, e.g. by using the density
or orbitals from the checkpoint file. For example:

In [5]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-2, checkpointargs...);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   |Magn|   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ------   ----   ----   ------
  1   -27.64662958012                   -0.13    0.001    0.188   0.80    6.5    264ms
  2   -28.92461206964        0.11       -0.83    0.691    0.696   0.80    2.0    1.24s
  3   -28.93116899657       -2.18       -1.13    1.197    1.207   0.80    2.0    133ms
  4   -28.93778418560       -2.18       -1.19    1.779    1.799   0.80    1.5    110ms
  5   -28.93917721967       -2.86       -1.02    2.000    2.061   0.80    2.0    103ms
  6   -28.93958657876       -3.39       -1.89    1.975    2.012   0.80    1.5    103ms
  7   -28.93961182589       -4.60       -3.15    1.985    2.023   0.80    1.0   91.9ms


Notice that the `ρ` argument is now passed to kwargs_scf_checkpoints instead.
If we run in the same folder the SCF again (here using a tighter tolerance),
the calculation just continues.

In [6]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-3, checkpointargs...);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   |Magn|   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ------   ----   ----   ------
  1   -28.93961293103                   -3.37    1.986    2.023   0.80   10.5    227ms


Since only the density is stored in a checkpoint
(and not the Bloch waves), the first step needs a slightly elevated number
of diagonalizations. Notice, that reconstructing the `checkpointargs` in this second
call is important as the `checkpointargs` now contain different data,
such that the SCF continues from the checkpoint.
By default checkpoint is saved in the file `dftk_scf_checkpoint.jld2`, which can be changed
using the `filename` keyword argument of `kwargs_scf_checkpoints`. Note that the
file is not deleted by DFTK, so it is your responsibility to clean it up. Further note
that warnings or errors will arise if you try to use a checkpoint, which is incompatible
with your calculation.

We can also inspect the checkpoint file manually using the `load_scfres` function
and use it manually to continue the calculation:

In [7]:
oldstate = load_scfres("dftk_scf_checkpoint.jld2")
scfres   = self_consistent_field(oldstate.basis, ρ=oldstate.ρ, ψ=oldstate.ψ, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   |Magn|   Diag   Δtime
---   ---------------   ---------   ---------   ------   ------   ----   ------
  1   -28.93840551077                   -2.22    1.986    2.019    5.0    113ms
  2   -28.93939144979       -3.01       -2.68    1.985    2.020    1.0    112ms
  3   -28.93960810490       -3.66       -2.46    1.985    2.022    4.0    131ms
  4   -28.93960980819       -5.77       -2.57    1.985    2.022    1.0   82.2ms
  5   -28.93961190913       -5.68       -2.79    1.985    2.023    1.0   82.8ms
  6   -28.93961199938       -7.04       -2.76    1.985    2.023    1.0   79.0ms
  7   -28.93961210826       -6.96       -2.77    1.985    2.023    1.0   83.9ms
  8   -28.93961219869       -7.04       -2.78    1.985    2.023    1.0   81.4ms
  9   -28.93961306331       -6.06       -3.13    1.985    2.023    1.0   81.2ms
 10   -28.93961308354       -7.69       -3.11    1.985    2.023    1.0   81.2ms
 11   -28.93961307017   +   -7.87       -

Some details on what happens under the hood in this mechanism: When using the
`kwargs_scf_checkpoints` function, the `ScfSaveCheckpoints` callback is employed
during the SCF, which causes the density to be stored to the JLD2 file in every iteration.
When reading the file, the `kwargs_scf_checkpoints` transparently patches away the `ψ`
and `ρ` keyword arguments and replaces them by the data obtained from the file.
For more details on using callbacks with DFTK's `self_consistent_field` function
see Monitoring self-consistent field calculations.

(Cleanup files generated by this notebook)

In [8]:
rm("dftk_scf_checkpoint.jld2")
rm("scfres.jld2")